In [92]:
%reset
import sys
import numpy as np
import bokeh.plotting as bkp
import bokeh.models as bkm
from bokeh.io import push_notebook
from ipywidgets import widgets
from ipywidgets import interact,interactive
from IPython import display
bkp.output_notebook()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


Loading BokehJS ...

# Solidification week 3 homework

### Mikko Karkkainen 11740378



The zone melting process is used to purify materials by taking advantage of the segregation of impurities to one end of the sample in directional solidification, as seen in Fig. 5.6. In zone melting, a segment of the sample of thickness $ \delta $ is melted, for example using an induction heater. The coil then traverses the sample beginning at one end. The heater produces sufficient stirring in the melted section that its composition can be considered to be uniform. The composition profile at some time during the first pass is shown schematically in the figure below. The initial composition of the sample is $ C_0 $, and the composition in the liquid is designated $ C_l^* $.

<img src="https://dev-plebbit.s3.amazonaws.com/uploads/topic/picture/17/zone_melting.png"/>
---

#### a) Write a solute balance for the change in composition in the liquid when the interface moves by an increment of $ dx $ as shown. Be sure to account for both the solute rejected at the solid-liquid-interface, and the dilution associated with previously unmelted solid.

Assuming no convection in the liquid, flux balance at the interface gives

$$ (C_0-C_S^*)dx=\delta \: dC_l^* $$

#### b) Rearrange this equation so that it can be integrated from the initial condition, where the composition in the liquid is $ C_0 $ and the solid-liquid interface is located at $ x^* = 0 $.


Let's rearrange:

$$  dx = \frac{\delta}{C_0 -C_S^*} \: dC^*_l $$ 

Substitute $ k_0C_l^* $ for  $ C_S^* $:

$$  dx = \frac{\delta}{C_0 - k_0C_l^*} \: dC^*_l $$

Define integral:

$$ \int^{x^*}_0 dx = \int_{C_0}^{C_l^*}\frac{\delta}{C_0 - k_0C_l^*} \: dC^*_l $$

---

#### c) Perform the integration to obtain the result

$$ \frac{C_l^*}{C_0} = \frac{1}{k_0} \Bigg[ 1 - (1-k_0)exp \bigg(-\frac{k_0x^*}{\delta} \bigg) \Bigg]$$

Performing the integral:

$$ x^* = - \frac{\delta}{k_0} \Big[ ln(C_0-k_0C_l^*) - ln(C_0-k_0C_0) \Big] $$

Rearranging:

$$ - \frac{k_0x^*}{\delta} = ln \Bigg( \frac{C_0-k_0C_l^*}{C_0-k_0C_0} \Bigg) $$

exp from both sides:

$$ exp \Big( - \frac{k_0x^*}{\delta} \Big) = \frac{C_0-k_0C_l^*}{C_0-k_0C_0}  $$

Rearrange:

$$ exp \Big( - \frac{k_0x^*}{\delta} \Big) = \frac{1-\frac{k_0C_l^*}{C_0}}{1-k_0}  $$

Rearrange:

$$ (1-k_0)exp \Big( - \frac{k_0x^*}{\delta} \Big) = 1-\frac{k_0C_l^*}{C_0}  $$

Rearrange:

$$ \frac{k_0C_l^*}{C_0}  = 1- (1-k_0)exp \Big( - \frac{k_0x^*}{\delta} \Big) $$

Rearrange:

$$ \frac{C_l^*}{C_0}  = \frac{1}{k_0} \Big[ 1- (1-k_0)exp \Big( - \frac{k_0x^*}{\delta} \Big) \Big] $$

---

#### d) Real zone melting processes perform multiple passes. One cannot find an analytical solution for the later passes. Sketch the solute profile that you would expect to see after the second and tenth passes. What would happen if this were a multicomponent aloy, whose constituents had different segregation coefficients.

Let's make a visualisation of the zone melting process. Let
s use the data for Al- 4.5wt%Cu from table B2, p. 387

We have the following equation for the solid concentration at the solidification interface:

$$ C_S^*  =  C_0 \Big[ 1- (1-k_0)exp \Big( \frac{k_0x^*}{\delta} \Big) \Big] $$

And the following equation for the liquid concentration at the solidification interface:

$$ C_l^*  = \frac{C_0}{k_0} \Big[ 1- (1-k_0)exp \Big( \frac{k_0x^*}{\delta} \Big) \Big] $$


In [93]:
C_0=0.045
k=0.14
delta=10
l=100
C_E=0.3217
C_SM=0.0565
C_L=C_0



# Initial state: liquid nodes, with concentration C_0

length_vector=np.linspace(0,l,l)
concentration_vector=np.empty([l])
concentration_vector.fill(C_0)

grid=100
step= l/grid
x_vector=np.linspace(0,l,grid)

C_L_vector_numeric=np.empty([grid])
C_L_vector_numeric.fill(C_0)
C_S_vector_numeric=np.empty([grid])
C_S_vector_numeric.fill(k*C_0)



#numeric functions for determining concentration:


def C_L_numeric(i):
    return (C_0-k*i)/delta

def C_S_numeric(i):
    return k*(C_0-i)/delta
    


#Analytic functions for determining  concentration:

def dx(a):
    return delta/(C_0-k*(C_0+i*step))


def C_S_analytic(a):
    return C_0*(1-(1-k)*np.exp(-k*a/delta))

def C_L_analytic(a):
    return C_0/k*(1-(1-k)*np.exp(-k*a/delta))


# numeric solution for liquid concentration:
for i in range(grid-1):
    C_L_vector_numeric[i+1]=C_L_vector_numeric[i]+step*C_L_numeric(C_L_vector_numeric[i])

# numeric solution for solid concentration:
for i in range(grid-1):
    C_S_vector_numeric[i+1]=C_S_vector_numeric[i]+step*C_S_numeric(C_S_vector_numeric[i])
    
f=bkp.figure(title="test",height=300,width=500)

# Analytic solutions:
f.line(length_vector,C_S_analytic(length_vector),legend="C_S analytic")
f.line(length_vector,C_L_analytic(length_vector),color="red",legend="C_L analytic")

# Numerical solutions plotted
f.scatter(x_vector,C_S_vector_numeric, fill_alpha=0, legend='C_S numeric')
f.scatter(x_vector,C_L_vector_numeric,fill_alpha=0,color='red',legend='C_L numeric')


# C_0 level plotted
a=np.empty(l)
a.fill(C_0)
f.line(length_vector,a,color="purple",legend="C_0")

bkp.show(f)

However, this representation lacks the final transient. We can use the Scheil equation to approximate it:

$$ C_S=kC_0(1-f_S)^{k-1} $$

While we're at it, let's also make a numerical solver for Scheil using the flux balance condition:

$$ (C_L^*-C_S^*)dfs=(1-f_S)dC_L $$

In [94]:
scheil_grid=100
scheil_portion= delta/grid*l
scheil_step=scheil_portion/scheil_grid

#Analytic Scheil::

def f_s(a):
    return a/l

def scheil_S_analytic(a):
    result= k*C_0*(1-a)**(k-1)
    #return np.where(result<=C_SM,result,C_SM)
    return result

def scheil_L_analytic(a):
    result= C_0*(1-a)**(k-1)
    #return np.where(result<=C_E,result,C_E) 
    return result

#Numerical Scheil:

def scheil_S_numeric(a,b):
    return k*(a/k-a)/(l-b)

def scheil_L_numeric(a,b):
    return (a-k*a)/(l-b)

#Solver:
scheil_x=np.linspace(l-scheil_portion,l,scheil_grid+1)
scheil_L=np.empty(scheil_grid)
scheil_S=np.empty(scheil_grid)

scheil_L[0]=scheil_L_analytic(scheil_x[0]/l)
scheil_S[0]=scheil_S_analytic(scheil_x[0]/l)

#Solid C
for i in range(len(scheil_S)-1):
    update=scheil_S[i]+scheil_step*scheil_S_numeric(scheil_S[i],l-scheil_portion+(i+1)*scheil_step)
    #if (update<=C_SM):
    scheil_S[i+1]=update
    #else:
    #    scheil_S[i+1]=C_SM

#Liquid C
for i in range(len(scheil_L)-1):  
    update=scheil_L[i]+scheil_step*scheil_L_numeric(scheil_L[i],l-scheil_portion+(i+1)*scheil_step)
    #if (update<=C_E):
    scheil_L[i+1]=update
    #else:
     #   scheil_L[i+1]=C_E
    
f2=bkp.figure()
f2.line(scheil_x,scheil_S_analytic(scheil_x/l),legend="S analytic")
f2.line(scheil_x,scheil_L_analytic(scheil_x/l),color="red",legend="L analytic")
f2.scatter(scheil_x,scheil_S,fill_alpha=0,legend="S numerical")
f2.scatter(scheil_x,scheil_L,color="red",fill_alpha=0,legend="L numerical")
bkp.show(f2)


Now let's make a model that combines these two, and accepts user defined values. First let's configure gui for user inputs:

In [104]:
#widget functions:
def k_func(k_0):
    return k_0
def C_0_func(C_0):
    return C_0
def l_func(length):
    return length
def delta_func(melt_nodes):
    return melt_nodes
def grid_func(grid_nodes):
    return grid_nodes
def pass_func(pass_count):
    return pass_count

# widgets using ipywidgets interactive:
k=widgets.interactive(k_func,k_0=widgets.FloatText(0.14))
C_0=widgets.interactive(C_0_func,C_0=widgets.FloatText(0.045))
l=widgets.interactive(l_func,length=widgets.FloatText(100))
delta=widgets.interactive(delta_func,melt_nodes=widgets.IntText(10))
grid=widgets.interactive(grid_func,grid_nodes=widgets.IntText(100))
pass_count=widgets.interactive(pass_func,pass_count=widgets.Dropdown(options=[2,3,4,5,6,7,8,9,10]))


#display ipywidgets.interactives
display.display(k)
display.display(C_0)
display.display(l)
display.display(delta)
display.display(grid)
display.display(pass_count)



2

Next let's define the functions and variables to perform the calculations

In [96]:
#Define global constants:

#step size between grid nodes:
step=l.result/grid.result

#Initialize *liquid* concentration matrix:
C_L=np.empty([grid.result,pass_count.result+1])
C_L.fill(C_0.result)

#Initialize *solid* concentration matrix:
C_S=np.empty([grid.result,pass_count.result+1])
C_S.fill(C_0.result)

#Initialize *visualisation* x-axis vector
x_values=np.linspace(0,l.result,grid.result)

#Functions for calculating liquid and solid concentration:

#Before final transient, change in liquid solute is the difference between
# concentration of old solid (which melts) and newly solidified solid :

def C_L_zone(new_solid,old_solid):
    return (old_solid-new_solid)/delta.result

# During final transient,use the scheil equation for liquid concentration:

def Scheil_L(new_liquid,s_fraction):
    return (new_liquid-k.result*new_liquid)/(l.result-s_fraction)

# Concentration of solid is given by k*C_L:

def C_S_function(C_L):
    return k.result*C_L


#Calculate solid and liquid concentrations for all passes

#for each pass:
for n in range(1,pass_count.result+1):

    #initialize the first delta liquid nodes and the first solid node:
     
    C_L_init=np.average(C_S[0::delta.result,n-1])
    for i in range(delta.result):
        C_L[i,n]=C_L_init
    C_S[0,n]=k.result*C_L[0,n]
            
    #for each grid node:
    for x in range(1,grid.result):
        # Check that we are not in final transient:
        if (x<=grid.result-delta.result):
            #solid concentration update:
            C_S[x,n]=C_S_function(C_L[x-1,n])
            #liquid concentration update:
            C_L[x,n]=C_L[x-1,n]+step*C_L_zone(C_S[x,n],C_S[x+delta.result-1,n-1])
            
        #C_S[x+delta.result-1,n-1]        
        elif (x<grid.result):
            #solid concentration:
            C_S[x,n]=k.result*C_L[x-1,n]
            #liquid concentration:
            C_L[x,n]=C_L[x-1,n]+step*Scheil_L(C_L[x-1,n],step*x)

In [102]:
C=np.empty(grid.result-1)
n=0

def initialize_stuff(rep):
    for i in range(grid.result-1):
        C[i]=C_S[i,rep-1]
    global n
    n=rep
    push_notebook(handle=t)
    
def update_plot(x):
            
    if (x<grid.result-delta.result):
        
        #solid concentration update:
        C[x]=C_S[x,n]

        if (x != 0):
            #liquid concentration update:
            for i in range(1,delta.result-1):
                C[x+i]=C_L[x,n]
            
            
    #during final transient:
    else:      
        #solid concentration:
        C[x]=C_S[x,n]
        #liquid concentration:
        for i in range(1,grid.result-x-1):
            C[x+i]=C_L[x,n]
    
    push_notebook(handle=t)


f=bkp.figure(title="Zone Melting",height=300,width=500)  
f.line(x_values,C)
#f.line(x_values,C_0.result*(1-(1-k.result)*np.exp(-k.result*x_values/delta.result)),color='red')
t=bkp.show(f, notebook_handle=True)

In [103]:
initialize=interact(initialize_stuff,rep=widgets.IntSlider(value=1,min=1,max=pass_count.result-1))


In [91]:
play=interact(update_plot,x=widgets.Play(value=0,min=0,max=grid.result))